In [1]:
import numpy as np

In [15]:
def SNG_10(seed: int, target: float,
        bitstream_length=1024, bit1=7, bit2=10, isprint=False):

    # check the seed in range (0-1023)
    assert seed > 0 and seed < 1023, "seed should be in range (0-1023)"
    # check the target in range (-1, 1)
    assert target >= -1 and target <= 1, "target should be in range (-1, 1)"
    
    # convert seed to binary
    seed_bin_str = format(seed, '010b')
    seed = np.array([int(i) for i in seed_bin_str])

    # convert target to unipolar
    target = (target + 1) / 2 # (-1, 1) -> (0, 1)
    target_10 = target * 1023

    lfsr = seed.copy()
    output_stream = np.zeros(bitstream_length)

    # do xor on bit1 and bit2
    for i in range(bitstream_length):
        xorbit = lfsr[10-bit1] ^ lfsr[10-bit2]  # xor on bit7 and bit10
        lfsr = np.roll(lfsr, -1)                # shift left
        lfsr[9] = xorbit                        # set xorbit to bit0
        lfsr_val = int(''.join([str(i) for i in lfsr]), 2) # array->binary->int
        if isprint: print(lfsr_val)
        if lfsr_val <= target_10:
            output_stream[i] = 1
        else:
            output_stream[i] = 0

    return output_stream

def SNG_10_matrix(seed_matrix: np.array, target_matrix: np.array,
                  bitstream_length=3, bit1=7, bit2=10, isprint=False):

    lfsr_matrix = seed_matrix.copy()
    target_matrix = (target_matrix + 1) / 2 * 1023

    output_stream_matrix = np.zeros((seed_matrix.shape[0], seed_matrix.shape[1], bitstream_length), dtype=np.int8)

    # Create weights matrix to convert binary matrix to integers
    weights = np.array([2**i for i in reversed(range(seed_matrix.shape[2]))])
    # if isprint: print('weights: \n',weights) # [512 256 128  64  32  16   8   4   2   1]

    for i in range(bitstream_length):
        xorbits = lfsr_matrix[:,:, 10-bit1] ^ lfsr_matrix[:,:, 10-bit2]
        lfsr_matrix = np.roll(lfsr_matrix, -1, axis=2)
        lfsr_matrix[:,:, 9] = xorbits

        # Matrix multiplication to get integer values
        lfsr_values = (lfsr_matrix * weights).sum(axis=2)

        if isprint: print(lfsr_values)
        
        # Set output_stream_matrix values based on the condition
        output_stream_matrix[:,:, i] = (lfsr_values <= target_matrix).astype(np.int8)

    return output_stream_matrix

    

import numpy as np
import time

def test_methods():
    seed_matrix = np.random.randint(0, 2, (8, 8, 10))
    target_matrix = np.random.uniform(0, 1, (8, 8))
    print(seed_matrix[0,0,:])
    print(target_matrix[0,0])


    SNG_10_matrix(seed_matrix, target_matrix,isprint=True)

test_methods()



[1 1 1 0 0 0 0 0 0 0]
0.9044602409667764
weights: 
 [512 256 128  64  32  16   8   4   2   1]
[[ 769  569  467  806   10 1011  200  208]
 [ 676  214   61  546  133   85  349  177]
 [ 953  223  872  108  567  295  198  532]
 [ 900  277  190   23  120  118  801  162]
 [ 665  960  503  663  709  799  913  428]
 [ 456  358  139  174  633  345  214  875]
 [ 365  902  327  767  876    9  437  341]
 [ 561  775   96  637  166  494  437  632]]
[[ 515  115  935  589   20  998  401  417]
 [ 329  429  122   69  266  171  699  354]
 [ 883  447  720  217  111  590  397   41]
 [ 777  554  380   46  241  237  579  324]
 [ 307  896 1007  303  394  575  803  856]
 [ 913  717  278  348  242  691  429  726]
 [ 731  781  655  510  728   18  874  683]
 [  99  527  193  250  332  989  874  240]]
[[   7  231  847  154   40  972  802  834]
 [ 659  858  245  139  532  342  375  709]
 [ 742  894  416  435  223  156  794   82]
 [ 531   85  761   92  483  475  134  649]
 [ 614  769  990  606  788  127  583  688]
 